# Data cleaning from meteorological data - UFC Campus Quixadá
- This notebook present the data cleaning process process described in the following paper: Time Series Forecasting for Purposes of Irrigation Management Process.

Libs required

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [17]:
exp = pd.read_csv('dataset.csv', index_col=['data'], sep=',',parse_dates=['data'])

In [18]:
exp.head(2)

,pressao_atm_max,pressao_atm_min,chuva_mm,temp_ar_max,temp_ar_min,umid_relat_max,umid_relat_min,rad_solar_total,temp_max,temp_min,temp_ar_media,umid_relat_media,vento_veloc,rad_solar_media,et0
data,,,,,,,,,,,,,,,
2017-01-01 00:00:00,619.9329,599.9176,0.0,25.936,25.426,81.828,78.974,1601.449,22.16899,22.00570,25.66374,80.65533,0.864792,4.448471,0.016252
2017-01-01 01:00:00,620.2747,599.7497,0.0,25.447,25.094,84.541,81.796,1566.530,22.36811,22.07685,25.24476,83.31339,0.746875,4.351471,0.012721


In [19]:
(exp['et0']).describe()

count    7948.000000
mean        0.043039
std         0.046288
min         0.000055
25%         0.003839
50%         0.023647
75%         0.071558
max         0.209839
Name: et0, dtype: float64

Removing outliers

In [20]:
exp = exp[(exp['chuva_mm'] <= 60)]
exp = exp[(exp['temp_min'] >= 0)]
exp = exp[(exp['umid_relat_min'] >= 20)]
exp = exp[(exp['rad_solar_total'] <= 4000)]

Downsampling to daily basis

In [21]:
exp = exp.resample(rule='D').mean()

Spliting train and test - 80/20

In [22]:
X = exp[[c for c in exp.columns if c != 'et0']]
y = exp[['et0']]

In [23]:
X_train, X_test = X['2017-01-01':'2017-09-24'], X['2017-09-24':'2017-11-29']
y_train, y_test = y['2017-01-01':'2017-09-24'], y['2017-09-24':'2017-11-29']

In [24]:
X_train.to_csv('./data/features_treino_completo.csv')
y_train.to_csv('./data/et0_treino_completo.csv')
X_test.to_csv('./data/features_teste.csv')
y_test.to_csv('./data/et0_teste.csv')

Split train in train and validation

In [25]:
X_train_, X_val = X_train[:213], X_train[213:]
y_train_, y_val = y_train[:213], y_train[213:]

In [26]:
X_train_.to_csv('./data/features_treino.csv')
y_train_.to_csv('./data/et0_treino.csv')
X_val.to_csv('./data/features_val.csv')
y_val.to_csv('./data/et0_val.csv')